<a href="https://colab.research.google.com/github/AlexsanderPessoa/DS_Fifa_Word_Cup_Catar_2022/blob/main/DS_Fifa_Word_Cup_Catar_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import pandas as pd
import numpy as np

#estamos utilizando poisson, pois ela ajuda quando o problema é de contagem
#Ex: quantas pessoas vão ligar no dia
#Ex: quantos carros vai passar no pedágio ...
from scipy.stats import poisson

#index_col = 0, significa que estou substituindo o índice pela coluna 0, agora cada seleção é seu proprio índice
selecoes = pd.read_excel('/content/DadosCopaDoMundoQatar2022.xlsx', sheet_name='selecoes', index_col = 0)

jogos = pd.read_excel('/content/DadosCopaDoMundoQatar2022.xlsx', sheet_name='jogos')

In [76]:
selecoes.head()

,Grupo,GrupoNumero,NomeEmIngles,PosiçãoRankingFIFA,ValorDeMercado,PontosRankingFIFA,Confederação,Copas,JogadorDestaque,FotoJogadorDestaque,LinkBandeiraPequena,LinkBandeiraGrande
Seleção,,,,,,,,,,,,
Catar,A,A1,Qatar,48,18.33,1442,AFC,0,Almoez Ali,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Equador,A,A2,Ecuador,44,125.80,1464,UEFA,0,Pervis Estupiñán,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Senegal,A,A3,Senegal,18,286.50,1585,CAF,0,Sadio Mané,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Holanda,A,A4,Netherlands,8,455.75,1679,UEFA,0,Virgil Van Dijk,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Inglaterra,B,B1,England,5,1360.00,1737,UEFA,1,Harry Kane,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...


In [77]:
jogos.head()

,data,hora,grupo,seleção1,seleção2,estádio
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR


In [78]:
# simulando uma partida
# no size é o tamanho de partidas que quero simular
# (lam) O número esperado de eventos ocorrendo em um intervalo de tempo fixo deve ser >= 0. Uma sequência deve ser difundida no tamanho solicitado.
np.random.poisson(lam = 1, size = 1)

# o resultado refere-se que se eu tivesse simulado os gols de uma equipe em uma partida, na qual a equipe tem em média de um gol por partida
# o resultado é o numero de gols

array([1])

In [79]:
# estou pegando o peso de cada seleção
fifa = selecoes['PontosRankingFIFA']

#entretanto preciso fazer uma transformação relacionado com as médias
a,b = min(fifa), max(fifa)

# fa e fa são os novos valores, o python vai fazer que os valores dos raking fique entre esses valores
# transformação linear de escala numerica
# 0,15 porque eu quero que o melhor seja no máximo 7x melhor
fa, fb = 0.15, 1

b1 = (fb - fa)/(b-a)
b0 = fb - b*b1
forca = b0 + b1*fifa

forca

Seleção
Catar             0.243596
Equador           0.285618
Senegal           0.516742
Holanda           0.696292
Inglaterra        0.807079
Irã               0.467079
Estados Unidos    0.612247
País de Gales     0.511011
Argentina         0.872022
Arábia Saudita    0.232135
México            0.640899
Polônia           0.442247
França            0.860562
Dinamarca         0.669551
Tunísia           0.369663
Austrália         0.323820
Espanha           0.768876
Alemanha          0.658090
Japão             0.459438
Costa Rica        0.354382
Bélgica           0.969438
Canadá            0.304719
Marrocos          0.465169
Croácia           0.606517
Brasil            1.000000
Sérvia            0.449888
Suíça             0.585506
Camarões          0.325730
Portugal          0.696292
Gana              0.150000
Uruguai           0.623708
Coreia do Sul     0.404045
Name: PontosRankingFIFA, dtype: float64

In [80]:
def MediasPoisson(selecao1, selecao2):
    forca1 = forca[selecao1]
    forca2 = forca[selecao2]

    #definimos a média de gols como 2.75
    mgols = 2.75

    l1 = mgols*forca1/(forca1 + forca2)
    l2 = mgols - l1

    #l1 e l2 são as médias de cada selecao, vai me retornar dentro de uma lista
    return [l1, l2]

# funcao resultados, vitória, empate ou derrota
def Resultado(gols1, gols2):
    if gols1 > gols2:
        resultado = 'V'
    elif gols1 < gols2:
        resultado = 'D'
    else:
        resultado = 'E'
    return resultado


    #calculado os pontos
def Pontos(gols1, gols2):
    resultado = Resultado(gols1, gols2)
    if resultado == 'V':
        pontos1, pontos2 = 3, 0
    elif resultado == 'D':
        pontos1, pontos2 = 0, 3
    else: pontos1, pontos2 = 1, 1
    return [pontos1, pontos2, resultado]


In [81]:
Resultado(1, 2)

'D'

In [82]:
MediasPoisson('Brasil', 'França')

[1.478048191315901, 1.271951808684099]

In [85]:
def Jogo(selecao1, selecao2):
    l1, l2 = MediasPoisson(selecao1, selecao2) 

    # gols do time 1
    gols1 = int(np.random.poisson(lam = l1, size = 1))

    # gols do time 2
    gols2 = int(np.random.poisson(lam = l2, size = 1))

    saldo1 = gols1 - gols2
    saldo2 = -saldo1
    pontos1, pontos2, resultado = Pontos(gols1, gols2)
    placar = '{}x{}'.format(gols1, gols2)
    
    return [gols1, gols2, saldo1, saldo2, pontos1, pontos2, resultado, placar]

In [97]:
# pmf probabilty master function
# são valores que calculam os valores da função de poisson, para dada uma média e gols
# qual seria a probaalidade de fazer zero gols em uma média de dois gols (poisson.pmf(0, 2))
def Distribuicao(media):
    probs = []
    #até 7 por causa da alemanha e brasil
    for i in range(7):
        probs.append(poisson.pmf(i,media))
    # 1 menos tudo que faltou
    probs.append(1-sum(probs))
    return pd.Series(probs, index = ['0', '1', '2', '3', '4', '5', '6', '7+'])

In [99]:
Distribuicao(2.75)

0     0.063928
1     0.175802
2     0.241727
3     0.221583
4     0.152339
5     0.083786
6     0.038402
7+    0.022433
dtype: float64